# Input files pathes

In [ ]:
import pandas as pd

#Input Files Pathes
by_path=input("Enter BY file path")
by_path= by_path[1:len(by_path)-1]


def str_to_raw(s):
    raw_map = {8:r'\b', 7:r'\a', 12:r'\f', 10:r'\n', 13:r'\r', 9:r'\t', 11:r'\v'}
    return r''.join(i if ord(i) > 32 else raw_map.get(ord(i), i) for i in s)

str_to_raw(by_path)


# Input SAP file path

sap_path=input("Enter SAP file path")
sap_path= sap_path[1:len(sap_path)-1]

def str_to_raw(s):
    raw_map = {8:r'\b', 7:r'\a', 12:r'\f', 10:r'\n', 13:r'\r', 9:r'\t', 11:r'\v'}
    return r''.join(i if ord(i) > 32 else raw_map.get(ord(i), i) for i in s)

str_to_raw(sap_path)

test_path = by_path.rsplit('\\',1)[0]+'\\test.xlsx' #To test results


# New columns names
new_columns_by = {'Generic Item Number':'generic_code_by' , 'Trade Item Number':'trade_code_by' , 'Total Qty':'quantity_by' ,'Lot/Batch':'supplier_batch_by','Location':'location_by','prtnum':'generic_code_by' , 'revlvl':'trade_code_by' , 'untqty(Available)':'available_quantity_by' ,'untqty(Hold)':'hold_quantity_by' ,'lotnum':'supplier_batch_by' }
new_columns_s4 = {'Scientific Item':'generic_code_sap' , 'Material':'trade_code_sap' , 'Unrestricted':'quantity_sap' ,'Supplier Batch':'supplier_batch_sap','Batch':'sap_batches'}                                                                                 

# *BY Data*

In [2]:
df_by=pd.read_excel(by_path)

In [3]:
df_by.columns

Index(['Unnamed: 0', 'wh_id', 'prt_client_id', 'prtnum', 'revlvl', 'ftpcod',
       'inv_attr_str2', 'untqty(Available)', 'untqty(Hold)', 'status',
       'dstloc', 'dstlod', 'lotnum', 'mandte', 'expire_dte', 'Rcpt Dt'],
      dtype='object')

In [4]:
df_by = df_by.rename(columns = new_columns_by)

if 'quantity_by' not in df_by:
    df_by['quantity_by']= df_by['available_quantity_by']+df_by['hold_quantity_by']
    
# Remove Staged Batches     
staged = ['1RSTG001','1RSTG002','1RSTG003','1RSTG004','1RSTG005','1RSTG006','1RSTG007','1RSTG008','1RSTGC','1RSTG','BULK01']   
if 'location_by' in df_by: 
    df_by = df_by[~df_by['location_by'].isin(staged)]

## Asessing Data and Cleaning Data

#### Remove Leading Zeros From Supplier Batches

In [5]:
df_by['supplier_batch_cleaned_by']= df_by['supplier_batch_by'].str.lstrip('0')

#### Check Data Types

In [6]:
# Remove null before converting to integer
by_null = df_by[df_by['supplier_batch_cleaned_by'].isna()]
df_by= df_by[df_by['supplier_batch_cleaned_by'].notnull()]

df_by['generic_code_by'] = df_by['generic_code_by'].apply(int).astype(str)
df_by['trade_code_by'] = df_by['trade_code_by'].apply(int).astype(str)
df_by.head()

,Unnamed: 0,wh_id,prt_client_id,generic_code_by,trade_code_by,ftpcod,inv_attr_str2,available_quantity_by,hold_quantity_by,status,dstloc,dstlod,supplier_batch_by,mandte,expire_dte,Rcpt Dt,quantity_by,supplier_batch_cleaned_by
0,0,NaN,NUPCO-DMM,4110291700000,4110291700002,NaN,784020024,500,0,OK,A1AD08A016,DMM0149762,OC1077,20200301,20250228,2021-04-04,500,OC1077
1,1,NaN,NUPCO-DMM,4110292500000,4110292500001,NaN,785060623,69,0,OK,A1AD14B006,DMM0148578,LMP4C0221,20210201,20260228,2021-03-29,69,LMP4C0221
2,2,NaN,NUPCO-DMM,4110292500000,4110292500001,NaN,785060623,39,0,OK,A1AD06A008,DMM0164456,LMP4C0521,20210501,20260531,2021-07-06,39,LMP4C0521
3,3,NaN,NUPCO-DMM,4110292500100,4110292500101,NaN,785010026,78,0,OK,A1AD09A004,DMM0164385,LMP120521,20210501,20260531,2021-07-07,78,LMP120521
4,4,NaN,NUPCO-DMM,4110292500100,4110292500101,NaN,785010026,40,0,OK,A1AD11A008,DMM0138240,LMP121220,20201201,20251231,2021-02-03,40,LMP121220


## Calculations

In [7]:
by_tableau=df_by.groupby(['generic_code_by','trade_code_by', 'supplier_batch_cleaned_by','supplier_batch_by'], as_index=False).sum()

#### Deal With Duplicated cleaned_batch/batch

In [8]:
df_by_dup = by_tableau[by_tableau.duplicated(['supplier_batch_cleaned_by','trade_code_by'],keep=False)]
df_by_dup['supplier_batch_cleaned_by'] =df_by_dup['supplier_batch_by'] 

by_tableau = by_tableau.drop_duplicates(['supplier_batch_cleaned_by','trade_code_by'],keep=False)
by_tableau=pd.concat([by_tableau,df_by_dup],axis=0)

<ipython-input-8-04c668e61ba7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_by_dup['supplier_batch_cleaned_by'] =df_by_dup['supplier_batch_by']


## Export BY File

In [9]:
null_by_path = by_path.rsplit('\\',1)[0]+'\\null_by.xlsx'
tableau_path = by_path.rsplit('\\',1)[0]+'\\tableau_by.xlsx'

by_null.to_excel(null_by_path)
by_tableau.to_excel(tableau_path)

# *S4 SAP*

In [10]:
df_sap = pd.read_excel(sap_path)

In [11]:
df_sap.columns

Index(['Customer Material Number', 'Several Customer Codes',
       'Old material no.', 'Scientific Item', 'Material', 'Material Number',
       'Plnt', 'Customer (GA)', 'SLoc', 'Batch', 'Supplier Batch', 'Cont./PO',
       'Several Supplier Batches', 'Manuf. Dte', 'SLED/BBD', 'Unrestricted',
       'BUn'],
      dtype='object')

In [12]:
df_sap = df_sap.rename(columns = new_columns_s4 )
df_sap.head()

,Customer Material Number,Several Customer Codes,Old material no.,generic_code_sap,trade_code_sap,Material Number,Plnt,Customer (GA),SLoc,sap_batches,supplier_batch_sap,Cont./PO,Several Supplier Batches,Manuf. Dte,SLED/BBD,quantity_sap,BUn
0,133990271,NaN,NaN,4.227171e+12,4.227171e+12,MASK NASAL 000/OO PREMATR MP01510 DRAEGE,E2C1,120000.0,1000.0,1507222A,1507222A,COB,NaN,2015-07-30 00:00:00,2020-07-30 00:00:00,0,EA
1,137400105,NaN,NaN,4.227171e+12,4.227171e+12,RESUSCI MNUAL INFANT SPUR 335102000 AMBU,E2C1,120000.0,1000.0,1000416603,1000416603,5000000321,NaN,2020-07-01 00:00:00,2023-07-31 00:00:00,0,EA
2,137400105,NaN,NaN,4.227171e+12,4.227171e+12,RESUSCI MNUAL INFANT SPUR 335102000 AMBU,E2C1,120000.0,1000.0,1000297164,1000297164,5000000321,NaN,2019-09-21 00:00:00,2022-09-21 00:00:00,0,EA
3,137400105,NaN,NaN,4.227171e+12,4.227171e+12,RESUSCI MNUAL INFANT SPUR 335102000 AMBU,E2C1,120000.0,1000.0,0002342795,1000503001,5000000586,NaN,2021-03-30 00:00:00,2024-03-04 00:00:00,0,EA
4,137400105,NaN,NaN,4.227171e+12,4.227171e+12,RESUSCI MNUAL INFANT SPUR 335102000 AMBU,E2C1,120000.0,1000.0,030914,030914,COB,NaN,2014-09-30 00:00:00,2019-08-30 00:00:00,0,EA


## Asessing and Cleaning

#### Remove Leading Zeros from Supplier Batches

In [13]:
df_sap['supplier_batch_cleaned_sap'] = df_sap['supplier_batch_sap'].str.lstrip('0')

#### Change Data Types

In [14]:
# Remove null before converting to integer
sap_null = df_sap[df_sap['supplier_batch_cleaned_sap'].isna()]
df_sap= df_sap[df_sap['supplier_batch_cleaned_sap'].notnull()]

df_sap['generic_code_sap'] = df_sap['generic_code_sap'].apply(int).astype(str)
df_sap['trade_code_sap'] = df_sap['trade_code_sap'].apply(int).astype(str)
df_sap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25123 entries, 0 to 25133
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Customer Material Number    24831 non-null  object 
 1   Several Customer Codes      23908 non-null  object 
 2   Old material no.            5849 non-null   float64
 3   generic_code_sap            25123 non-null  object 
 4   trade_code_sap              25123 non-null  object 
 5   Material Number             25121 non-null  object 
 6   Plnt                        25123 non-null  object 
 7   Customer (GA)               25123 non-null  float64
 8   SLoc                        25123 non-null  float64
 9   sap_batches                 25123 non-null  object 
 10  supplier_batch_sap          25123 non-null  object 
 11  Cont./PO                    25046 non-null  object 
 12  Several Supplier Batches    4067 non-null   object 
 13  Manuf. Dte                  243

## Calculations

In [15]:
sap_tableau=df_sap.groupby(['generic_code_sap','trade_code_sap','supplier_batch_cleaned_sap','supplier_batch_sap'], as_index=False).agg({
                          'sap_batches' : lambda x: "[%s]" % ', '.join(x.astype(str)), 'quantity_sap' : ['sum']})

sap_tableau.columns = sap_tableau.columns.droplevel(0)
sap_tableau.columns= ['generic_code_sap','trade_code_sap','supplier_batch_cleaned_sap','supplier_batch_sap','sap_batches','quantity_sap']

In [16]:
sap_tableau.head()

,generic_code_sap,trade_code_sap,supplier_batch_cleaned_sap,supplier_batch_sap,sap_batches,quantity_sap
0,4010151200000,4010151200001,219854701,219854701,"[0000000279, 219854701]",0
1,4010151200000,4010151200001,219937932,219937932,"[219937932, 0000000280]",0
2,4010151200000,4010151200002,219175707,219175707,[0002361604],0
3,4110291700000,4110291700002,E2080,0E2080,[0002348055],0
4,4110291700000,4110291700002,OC1077,OC1077,[OC1077],500


#### Deal with Duplicated cleaned_batch/batch duplicate

In [17]:
df_sap_dup = sap_tableau[sap_tableau.duplicated(['supplier_batch_cleaned_sap','trade_code_sap'],keep=False)]

df_sap_dup['supplier_batch_cleaned_sap'] =df_sap_dup['supplier_batch_sap'] 

sap_tableau = sap_tableau.drop_duplicates(['supplier_batch_cleaned_sap','trade_code_sap'],keep=False)
sap_tableau=pd.concat([sap_tableau,df_sap_dup],axis=0)

<ipython-input-17-d2d559324e04>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sap_dup['supplier_batch_cleaned_sap'] =df_sap_dup['supplier_batch_sap']


## Export SAP Files

In [18]:
sap_tableau.to_excel(r"C:\Users\osama\Documents\nupco\stock adjustment\sap_grouped.xls")
df_sap_dup.to_excel(r"C:\Users\osama\Documents\nupco\stock adjustment\sap_duplicates.xls")

In [19]:
null_sap_path = by_path.rsplit('\\',1)[0]+'\\null_sap.xlsx'
tableau_path = by_path.rsplit('\\',1)[0]+'\\tableau_sap.xlsx'

by_null.to_excel(null_sap_path)
sap_tableau.to_excel(tableau_path)